## [실습] 문서 스캐너

* 문서스캐너

    - 카메라로 촬영한 문서 영상을 똑바로 펴서 저장해주는 프로그램
    
    
* 구현할 기능

    - 마우스로 문서 모서리 선택 & 이동하기
    - 키보드 ENTER키 인식
    - 왜곡된 문서 영상을 직사각형 형태로 똑바로 펴기 (투시 변환)
    
    
* 마우스로 문서 모서리 선택 & 이동하기

    - 마우스 왼쪽 버튼이 눌린 좌표가 네 개의 모서리와 근접해 있는지를 검사
    - 특정 모서리를 선택했다면 마우스 드래그를 검사
    - 마우스 드래그 좌표 이동 & 화면 표시
    - 마우스 왼쪽 버튼이 떼어졌을 때의 좌표를 기록


* 왜곡된 문서 영상을 직사각형 형태로 똑바로 펴기 (투시 변환)

    - 네 개의 모서리 좌표를 순서대로 srcQuad 배열에 추가
    - dstQuad 배열에는 미리 정의한 A4용지 크기의 네 모서리 좌표를 저장 (A4 용지 크기: 210x297cm)
    - srcQuad 점들로부터 dstQuad 점들도 이동하는 투시 변환 계산
    - 계산된 투시 변환 행렬을 이용하여 영상을 투시 변환하여 화면 출력

In [1]:
# 필요한 라이브러리를 로딩
import sys, os
import numpy as np
import cv2

In [2]:
# 이미지를 읽는다.



In [ ]:
#



[방법1]문서펴기

In [ ]:
import sys
import numpy as np
import cv2

def drawROI(img,corners):
    cpy=img.copy()

    c1=(192,192,255)          #1번컬러
    c2=(128,128,255)          #2번컬러

    for pt in corners:
        cv2.circle(cpy,tuple(pt),25,c1,-1,cv2.LINE_AA)
        
    cv2.line(cpy,tuple(corners[0]),tuple(corners[1]),c2,2,cv2.LINE_AA)
    cv2.line(cpy,tuple(corners[1]),tuple(corners[2]),c2,2,cv2.LINE_AA)
    cv2.line(cpy,tuple(corners[2]),tuple(corners[3]),c2,2,cv2.LINE_AA)
    cv2.line(cpy,tuple(corners[3]),tuple(corners[0]),c2,2,cv2.LINE_AA)

    disp=cv2.addWeighted(img,0.7,cpy,0.3,0)   #연산속도가 느림
    return disp
    
    #return cpy


def onMouse(event,x,y,flags,param):
    global srcQuad,dragSrc,ptOld,src

    if event==cv2.EVENT_LBUTTONDOWN:
        for i in range(4):
            if cv2.norm(srcQuad[i]-(x,y))<25:     #마우스가 원 안을 누를거리  // 드래그를 시작한다
                dragSrc[i]=True                    
                ptOld=(x,y)
                break

    if event == cv2.EVENT_LBUTTONUP:
        for i in range(4):
            dragSrc[i] = False

    if event == cv2.EVENT_MOUSEMOVE:
        for i in range(4):
            if dragSrc[i]:
                dx = x - ptOld[0]              #현재점 - 이전 점
                dy = y - ptOld[1]

                srcQuad[i] += (dx, dy)

                cpy = drawROI(src, srcQuad)    #이동한 결과를 화면에 보여주기
                cv2.imshow('img', cpy)
                ptOld = (x, y)
                break
    
src=cv2.imread('data/scan2.jpg')
src=cv2.resize(src,(1080,720))

if src is None:
    print('Image open failed')
    sys.exit()

#입력영상 크기 및 출력영상 크기
h,w= src.shape[:2]
dw=500
dh=round(dw*297/210)  #A4용지 크기: 210x297cm

#모서리 점들의 좌표, 드래그 상태 여부
srcQuad=np.array([[30,30],[30,h-30],[w-30,h-30],[w-30,30]],np.float32)
dstQuad=np.array([[0,0],[0,dh-1],[dw-1,dh-1],[dw-1,0]],np.float32)
dragSrc=[False,False,False,False]   #4개의 점중 어떤 점을 드래그 하고 있는지 상태정보


#모서리점, 사각형 그리기 
disp=drawROI(src,srcQuad)

cv2.imshow('img',disp)
cv2.setMouseCallback('img',onMouse)

while True:
    key=cv2.waitKey()
    if key==13:   #enter 키
        cv2.destroyWindow('img')
        break
    elif key==27:
        cv2.destroyWindow('img')
        sys.exit()

pers=cv2.getPerspectiveTransform(srcQuad,dstQuad)
dst=cv2.warpPerspective(src,pers,(dw,dh),flags=cv2.INTER_CUBIC)

cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

[방법2]문서펴기

In [ ]:
# 필요한 라이브러리를 로딩한다.
import cv2, os
import numpy as np

# 파일을 읽어온다.
img_path = 'data/scan2.jpg'
filename, ext = os.path.splitext(os.path.basename(img_path))
ori_img = cv2.imread(img_path)

# 원본 그림에서 불러올 이미지의 점들을 저장하는 변수
# 순서: 왼쪽위, 오른쪽위, 오른쪽아래, 왼쪽 아래 순서로 각점의 좌표
src = []


# mouse callback handler 함수

# 기본적으로 마우스 핸들러가 받는 파라미터는 이벤트, x,y값을 사용
def mouse_handler(event, x, y, flags, param):
    # 눌렀다 뗄떼만..
  if event == cv2.EVENT_LBUTTONUP:
    img = ori_img.copy()

    src.append([x, y])
#클릭할때마다 점에 동그라미 그리기
    for xx, yy in src:
      cv2.circle(img, center=(xx, yy), radius=5, color=(0, 255, 0), thickness=-1, lineType=cv2.LINE_AA)

    cv2.imshow('img', img)

    # 점4개 : 소스. 출력값
    # perspective transform
    if len(src) == 4:
      src_np = np.array(src, dtype=np.float32)

      width = max(np.linalg.norm(src_np[0] - src_np[1]), np.linalg.norm(src_np[2] - src_np[3]))
      height = max(np.linalg.norm(src_np[0] - src_np[3]), np.linalg.norm(src_np[1] - src_np[2]))

      dst_np = np.array([
        [0, 0],
        [width, 0],
        [width, height],
        [0, height]
      ], dtype=np.float32)

      M = cv2.getPerspectiveTransform(src=src_np, dst=dst_np)
      result = cv2.warpPerspective(ori_img, M=M, dsize=(width, height))

      cv2.imshow('result', result)
      cv2.imwrite('./result/%s_result%s' % (filename, ext), result)

# main 부분

#윈도우의 이름을 지정하고
cv2.namedWindow('img')
#마우스 콜백함수를 지정 --> 윈도우에서 감지되는 
#마우스 이벤트가 감지되면 마우스 핸들러로 전달
cv2.setMouseCallback('img', mouse_handler)

# 오리지널 이미지를 윈도우에 띄운다.
cv2.imshow('img', ori_img)
cv2.waitKey(0)

# 영상의  특징 추출

## 영상의 미분과 소베필터

* 에지(edge)

    - 영상에서 픽셀의 밝기 값이 급격하게 변하는 부분

    - 일반적으로 배경과 객체, 또는 객체와 객체의 경계
    
    <img src="data/eg1.png" width=400px>
    
    
* 기본적인 에지 검출 방법

    - 영상을 (x,y)변수의 함수로 간주했을 때, 이 함수의 1차 미분 값이 크게 나타나는 부분을 검출

    <img src="data/eg2.png" width=400px>
    
    
* 1차 미분의 근사화 (approximation)

     <img src="data/eg3.png" width=400px>
     
* 다양한 미분 마스크

    <img src="data/eg4.png" width=400px>

[소벨 필터를 이용한 미분 함수]

```cv2.Sobel(src,ddepth,dx,dy,dst=None,ksize=None,scale=None,delta=None,borderType=None)```

* 옵션값의 의미

    - src: 입력영상
    - ddepth: 출력영상데이터타입. -1이면 입력 영상과 같은 데이터 타입을 사용.
    - dx: x 방향 미분 차수.
    - dy: y 방향 미분 차수.
    - dst: 출력 영상(행렬)
    - ksize: 커널크기. 기본값은3.
    - scale: 연산 결과에 추가적으로 곱할값. 기본값은1.
    - delta: 연산 결과에 추가적으로 더할값. 기본값은0.(더하면 영상이 더 밝아진다)
    - borderType: 가장자리 픽셀 확장 방식. 기본값은cv2.BORDER_DEFAULT.
    - 대부분 dx=1, dy=0, ksize=3 또는 dx=0, dy=1, ksize=3 으로 지정

In [ ]:
import sys
import numpy as np
import cv2


src = cv2.imread('data/lena.png', cv2.IMREAD_GRAYSCALE)
src=cv2.resize(src,(512,512))

if src is None:
    print('Image load failed!')
    sys.exit()

dx=cv2.Sobel(src,-1,1,0,delta=128)
dy=cv2.Sobel(src,-1,0,1,delta=128)

cv2.imshow('dx',dx)
cv2.imshow('dy',dy)
cv2.waitKey()

## 그래디언트와 에지 검출

* 영상의 그래디언트 (gradient)

    - 함수 f(x,y)를 x축과 y축으로 각각 편미분하여 벡터 형태로 표현한 것
    
        <img src="data/gr1.png" width=400px>
        
        
* 실제 영상에서 구한 그래디언트 크기와 방향

    - 그래디언트 크기: 픽셀 값의 차이 정도, 변화량
    - 그래디언트 방향: 픽셀 값이 가장 급격하게 증가하는 방향
    
        <img src="data/gr2.png" width=400px>
        
* 2D 벡터의 크기 계산 함수

```cv2.magnitude(x,y,magnitude=None) -> magnitude```

   - 옵션
   
       - x: 2D 벡터의x 좌표행렬. 실수형.
       - y: 2D 벡터의y 좌표행렬. x와같은크기.실수형.
       - magnitude: 2D 벡터의크기행렬. x와같은크기, 같은타입.
       
       
* 2D 벡터의 방향 계산 함수

```cv2.phase(x,y,angle=None,angleInDegrees=None) -> angle```

   - 옵션
    
        - x: 2D 벡터의x 좌표행렬. 실수형.
        - y: 2D 벡터의y 좌표행렬. x와같은크기.실수형.
        - angle: 2D 벡터의크기행렬. x와같은크기, 같은타입.
        
            - angle(𝐼𝐼)=atan2(y𝐼𝐼,x𝐼𝐼)
            - 만약x(I)=y(I)=0이면angle은0으로설정됨.
            
        - angleInDegrees: True이면각도단위, False이면래디언단위.

In [ ]:
import sys
import numpy as np
import cv2

src=cv2.imread('data/lena.png',cv2.IMREAD_GRAYSCALE)
src=cv2.resize(src,(512,512))

if src is None:
    print('Image load failed!')
    sys.exit()

dx=cv2.Sobel(src,cv2.CV_32F,1,0)
dy=cv2.Sobel(src,cv2.CV_32F,0,1)

mag=cv2.magnitude(dx,dy)                 #255를 넘는 값들이 존재해서 하얗게 나옴
mag=np.clip(mag,0,255).astype(np.uint8)   #0~255 사이의 값으로 조정

dst=np.zeros(src.shape[:2],np.uint8)
dst[mag>80]=255                          

cv2.imshow('mag',mag)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 캐니 에지 검출

* 좋은 에지 검출기의 조건

* 정확한 검출 : 에지가 아닌 점을 에지로 찾거나 또는 에지인데 에지로 찾지 못하는 확률을 최소화

* 정확한 위치: 실제 에지의 중심을 검출

* 단일 에지: 하나의 에지는 하나의 점으로 표현

  <img src="data/cany1.png" width=600px>
  
  <img src="data/cany2.png" width=600px>

  <img src="data/cany3.png" width=600px>
  
  <img src="data/cany4.png" width=600px>
   
  <img src="data/cany5.png" width=600px>
  
  
* 케니 에지 검출 함수

```cv2.Canny(image,threshold1,threshold2,edges=None,apertureSize=None,L2gradient=None)```

* 옵션들

    - image: 입력영상
    - threshold1: 하단임계값 (threshold1 : threshold2 = 1:2 또는 1:3)
    - threshold2: 상단임계값
    - edges: 에지영상
    - apertureSize: 소벨연산을위한커널크기. 기본값은3.
    - L2gradient: True이면L2 norm 사용, False이면L1 norm 사용. 기본값은False

In [ ]:
import sys
import numpy as np
import cv2

src=cv2.imread('data/cake1.jpg',cv2.IMREAD_GRAYSCALE)
src=cv2.resize(src,(512,512))

dst=cv2.Canny(src,80,160)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 허프 변환: 직선 검출

* 허프 변환(Hough transform) 직선 검출

    - 2차원 영상 좌표에서의 직선의 방정식을 파라미터(parameter)공간으로 변환
    - 직선을 찾는 알고리즘

  <img src= "data/hf1.png" width=400px>
  
* 축적 배열(accumulation array)

    - 직선 성분과 관련된 원소 값을 1씩 증가시키는 배열
    
      <img src= "data/hf2.png" width=400px>
      
* 직선의 방정식 y=ax+b를 사용할 때의 문제점

    - y축과 평행한 수직선을 표현하지 못함 -> 극좌표계 직선의 방정식을 사용
    
         <img src= "data/hf3.png" width=400px> 
         
         <img src= "data/hf4.png" width=400px>
         
         
* 허브 변환에 읜한 선분 검출

```cv2.HoughLines(image,rho,theta,threshold,line=None,srn=None,stn=None,min_theta=None,max_theta=None)```

* 옵션들

    - image: 입력 에지 영상
    - rho: 축적배열에서rho 값의간격. (e.g.) 1.0 -> 1픽셀간격.
    - theta: 축적배열에서 theta 값의 간격 (e.g.) np.pi/180 -> 1도 간격
    - threshold: 축적 배열에서 직선으로 판단할 임계값
    
    - lines: 직선 파라미터 (rho,theta) 
         - 정보를 담고 있는 numpy.ndarray. shape=N,1,2).dtype=numpy.float32
         
    - srn, stn: 멀티 스케일 허프 변환에서 rho 해상도, theta 해상도를 나누는 값. 
         - 기본값은 0이고, 이 경우 일반 허프 변환 수행
         
    - min_theta, max_theta: 검출할 선분의 최소, 최대 theta 값
    
    
* 확률적 허프 변환에 의한 선분 검출

```cv2.HoughLinesP(image,rho,theta,threshold,line=None,minLineLength=None,maxLineGap=None)```

* 옵션들

    - image: 입력 에지 영상
    - rho: 축적배열에서rho 값의간격. (e.g.) 1.0 -> 1픽셀간격.
    - theta: 축적배열에서 theta 값의 간격 (e.g.) np.pi/180 -> 1도 간격
    - threshold: 축적 배열에서 직선으로 판단할 임계값
    - lines: 선분과 시작의 끝 좌표 (x1,y1,x2,y2) 정보를 담고 있는 numpy.ndarray. 
      shape=(N,1,4).dtype=numpy.int32
    - minLineLength: 검출할 선분의 최소 길이
    - maxLineGap: 직선으로 간주할 최대 에지 점 간격

In [ ]:
import sys
import numpy as np
import cv2


src = cv2.imread('building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

edges=cv2.Canny(src,50,150)
lines=cv2.HoughLinesP(edges,1.0,np.pi/180,160,minLineLength=50,maxLineGap=5)     
#rho 값이 작으면 축적배열이 커지고 rho 값이 크면 축적배열이 작아진다. 
# 축적배열이 작아지면 시간은 빠르지만 정밀하지가 않다
#360도로 나누면 정교해지만 느리다
#threshold는 축적배열에서 몇 이상부터 직선으로 판단할지 임계값이다. threshold가 낮으면 많은 직선이 검출될수 있다.
#lines은 선분의 시작과 끝 좌표(x1,y1,x2,y2) shape=(N,1,4) 중간값은 무시
dst=cv2.cvtColor(edges,cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):    #range(N)
        pt1=(lines[i][0][0],lines[i][0][1])
        pt2=(lines[i][0][2],lines[i][0][3])
        cv2.line(dst,pt1,pt2,(0,0,255),2,cv2.LINE_AA)


cv2.imshow('edges',edges)
print(lines)
cv2.imshow('dst',dst)
cv2.waitKey()

## 허프 원 변환: 원 검출

* 허프 변환을 응용하여 원을 검출할 수 있음

* 속도 향상을 위해 Hough gradient method 사용

    - 입력 영상과 동일한 2차원 평면 공간에서 축적 영상을 생성
    - 에지 픽셀에서 그래디언트 계산
    - 에지 방향에 따라 직선을 그리면서 값을 누적
    - 원의 중심을 먼저 찾고, 적절한 반지름 검출
    - 단점: 여러개의 동심원을 검출 못함 --> 가장 작은 원 하나만 검출됨
    
 <img src="data/hfwon.png" px=400>

In [ ]:
import sys
import numpy as np
import cv2
from skimage.data import coins

# 입력 이미지 불러오기
src = cv2.imread('dial.jpg')

if src is None:
    print('Image open failed!')
    sys.exit()

gray=cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
blr=cv2.GaussianBlur(gray,(0,0),1.0)          #cv2.Hough.GRADIENT 방법 사용시 블러링 권장

def on_trackbar(pos):
    rmin=cv2.getTrackbarPos('minRadius','img')
    rmax=cv2.getTrackbarPos('maxRadius','img')
    th=cv2.getTrackbarPos('threshold','img')

    circles=cv2.HoughCircles(blr,cv2.HOUGH_GRADIENT,1,50,
                            param1=120,param2=th,minRadius=rmin,
                            maxRadius=rmax)

    dst=src.copy()
    if circles is not None:
        for i in range(circles.shape[1]):
            cx,cy,radius=circles[0][i]
            cv2.circle(dst,(cx,cy),int(radius),(0,0,255),2,cv2.LINE_AA)
    
    cv2.imshow('img',dst)

cv2.imshow('img',src)
cv2.createTrackbar('minRadius','img',0,100,on_trackbar)
cv2.createTrackbar('maxRadius','img',0,150,on_trackbar)
cv2.createTrackbar('threshold','img',0,100,on_trackbar)
cv2.setTrackbarPos('minRadius','img',10)
cv2.setTrackbarPos('maxRadius','img',80)
cv2.setTrackbarPos('threshold','img',40)
cv2.waitKey()
cv2.destroyAllWindows()

## [실습] 동전 카운터

*  영상의 동전을 검출하여 금액이 얼마인지를 자동으로 계산하는 프로그램입니다.

* 편의상 동전은 100원짜리와 10원짜리만 있다고 가정했습니다.

* 구현할 기능

    - 동전 검출하기 -> 허프 원 검출

    - 동전 구분하기 -> 색상 정보 이용


* 색상 정보를 이용할 때 hsv로 색 공간을 변환해서 h값을 이용합니다.

* 10원 동전의 경우 빨강색 성분이 있어 히스토그램 값을 보았을 때 h값이 양쪽에 위치하게 됩니다.

* 양쪽에 있게 되면 10원을 판별하기가 어렵습니다.

 

* 이를 해결하기 위해 동전 영역 픽셀에 대해 hue값을 +40만큼 시프트하고, hue 평균을 이용했습니다.

* hue 평균이 90보다 작으면 10원, 90보다 크면 100원 입니다.